In [ ]:
from acquire_zillow import save_df

save_df()

In [ ]:
from acquire_mall import get_mall_data
df = get_mall_data()

In [ ]:
%matplotlib inline

In [ ]:
df = df.set_index('customer_id')
df.head(10)

In [ ]:
df.gender.value_counts().plot.bar()

In [ ]:
df.age.plot.hist()

In [ ]:
df.annual_income.plot.hist()

In [ ]:
df.spending_score.plot.hist()

In [ ]:
def get_upper_outliers(s, k):
    '''
    Given a series and a cutoff value, k, returns the upper outliers for the
    series.

    The values returned will be either 0 (if the point is not an outlier), or a
    number that indicates how far away from the upper bound the observation is.
    '''
    q1, q3 = s.quantile([.25, .75])
    iqr = q3 - q1
    upper_bound = q3 + k * iqr
    return s.apply(lambda x: max([x - upper_bound, 0]))

get_upper_outliers(df['annual_income'], 1.5)

In [ ]:
df.head(10)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode the gender using a sklearn label encoder.
encoder = LabelEncoder()
encoder.fit(df.gender)
df['gender_id'] = encoder.transform(df.gender)
df.gender_id

In [ ]:
df.head(10)

In [ ]:
from summarize import summarize_data

In [ ]:
summarize_data(df)

In [42]:
import pandas as pd

def join_zillow_data():
    # The FOUR HORSEMEN of the Apocalypse
    prop6_df = pd.read_csv('properties_2016.csv', low_memory=False)
    prop7_df = pd.read_csv('properties_2017.csv', low_memory=False)
    pred6_df = pd.read_csv('predictions_2016.csv', low_memory=False)
    pred7_df = pd.read_csv('predictions_2017.csv', low_memory=False)

    # Join predictions and properties together
    df6 = pd.merge(pred6_df, prop6_df, on='parcelid', how='inner')
    df7 = pd.merge(pred7_df, prop7_df, on='parcelid', how='inner')

    # Join 2016 and 2017 data together
    df = pd.concat([df6, df7])

    return df

df = join_zillow_data()

In [43]:
df.head(10)

,parcelid,logerror,transactiondate,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,...,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,airconditioningdesc,architecturalstyledesc,buildingclassdesc,propertylandusedesc,storydesc,typeconstructiondesc,heatingorsystemdesc
0,11016594,0.0276,2016-01-01,2931667,1.0,NaN,NaN,2.0,3.0,NaN,...,NaN,NaN,6.037107e+13,Central,NaN,NaN,Single Family Residential,NaN,NaN,Central
1,14366692,-0.1684,2016-01-01,475764,NaN,NaN,NaN,3.5,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,NaN
2,12098116,-0.0040,2016-01-01,2444575,1.0,NaN,NaN,3.0,2.0,NaN,...,NaN,NaN,6.037464e+13,Central,NaN,NaN,Single Family Residential,NaN,NaN,Central
3,12643413,0.0218,2016-01-02,1584730,1.0,NaN,NaN,2.0,2.0,NaN,...,NaN,NaN,6.037296e+13,Central,NaN,NaN,Condominium,NaN,NaN,Central
4,14432541,-0.0050,2016-01-02,2926916,NaN,NaN,NaN,2.5,4.0,NaN,...,NaN,NaN,6.059042e+13,NaN,NaN,NaN,Single Family Residential,NaN,NaN,NaN
5,11509835,-0.2705,2016-01-02,2843942,1.0,NaN,NaN,4.0,4.0,NaN,...,NaN,NaN,6.037621e+13,Central,NaN,NaN,Single Family Residential,NaN,NaN,Central
6,12286022,0.0440,2016-01-02,1628001,NaN,NaN,NaN,1.0,2.0,NaN,...,NaN,NaN,6.037542e+13,NaN,NaN,NaN,Single Family Residential,NaN,NaN,Floor/Wall
7,17177301,0.1638,2016-01-02,2543293,NaN,NaN,NaN,2.5,3.0,NaN,...,NaN,NaN,6.111003e+13,NaN,NaN,NaN,Condominium,NaN,NaN,NaN
8,14739064,-0.0030,2016-01-02,519006,NaN,NaN,NaN,1.0,2.0,NaN,...,NaN,NaN,6.059042e+13,NaN,NaN,NaN,Condominium,NaN,NaN,NaN
9,14677559,0.0843,2016-01-03,1604540,NaN,NaN,NaN,2.0,2.0,NaN,...,NaN,NaN,6.059063e+13,NaN,NaN,NaN,Condominium,NaN,NaN,NaN


In [44]:
# only include properties that include a latitude and longitude value
df = df.dropna(subset = ['latitude', 'longitude'])

In [45]:
df.latitude.isna().sum()

0

In [46]:
df.longitude.isna().sum()

0

In [47]:
df.shape

(167854, 68)

In [48]:
df['unitcnt'].value_counts(dropna=False)

 1.0      101916
NaN        58798
 2.0        4385
 4.0        1597
 3.0        1148
 6.0           2
 45.0          1
 11.0          1
 237.0         1
 42.0          1
 5.0           1
 9.0           1
 143.0         1
 70.0          1
Name: unitcnt, dtype: int64

In [52]:
# include only single unit properties (e.g. no duplexes, no land/lot, ...)
df = df.loc[(df['unitcnt'] == 1) | ((df['bathroomcnt'] >= 1) & (df['bathroomcnt'] <= 7))]

In [53]:
df['unitcnt'].value_counts(dropna=False)

 1.0      101916
NaN        57757
 2.0        4024
 4.0        1224
 3.0        1056
 6.0           2
 11.0          1
 143.0         1
 70.0          1
Name: unitcnt, dtype: int64

In [54]:
# takes in a dataframe and a list of columns names and returns the
# dataframe with the datatypes of those columns changed to a non-numeric type
def convert_to_string(df, *cols):
    for col in cols:
        df[col] = df[col].astype(str)
    return df

convert_to_string(df, 'id', 'airconditioningtypeid', 'architecturalstyletypeid', 'buildingclasstypeid',
                  'buildingqualitytypeid', 'decktypeid', 'fips', 'hashottuborspa', 'heatingorsystemtypeid',
                  'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'propertylandusetypeid', 'rawcensustractandblock',
                  'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 'storytypeid',
                  'typeconstructiontypeid', 'fireplaceflag', 'taxdelinquencyflag' ).head(10)

,parcelid,logerror,transactiondate,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,...,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,airconditioningdesc,architecturalstyledesc,buildingclassdesc,propertylandusedesc,storydesc,typeconstructiondesc,heatingorsystemdesc
0,11016594,0.0276,2016-01-01,2931667,1.0,nan,NaN,2.0,3.0,nan,...,nan,NaN,6.037107e+13,Central,NaN,NaN,Single Family Residential,NaN,NaN,Central
1,14366692,-0.1684,2016-01-01,475764,nan,nan,NaN,3.5,4.0,nan,...,nan,NaN,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,NaN
2,12098116,-0.0040,2016-01-01,2444575,1.0,nan,NaN,3.0,2.0,nan,...,nan,NaN,6.037464e+13,Central,NaN,NaN,Single Family Residential,NaN,NaN,Central
3,12643413,0.0218,2016-01-02,1584730,1.0,nan,NaN,2.0,2.0,nan,...,nan,NaN,6.037296e+13,Central,NaN,NaN,Condominium,NaN,NaN,Central
4,14432541,-0.0050,2016-01-02,2926916,nan,nan,NaN,2.5,4.0,nan,...,nan,NaN,6.059042e+13,NaN,NaN,NaN,Single Family Residential,NaN,NaN,NaN
5,11509835,-0.2705,2016-01-02,2843942,1.0,nan,NaN,4.0,4.0,nan,...,nan,NaN,6.037621e+13,Central,NaN,NaN,Single Family Residential,NaN,NaN,Central
6,12286022,0.0440,2016-01-02,1628001,nan,nan,NaN,1.0,2.0,nan,...,nan,NaN,6.037542e+13,NaN,NaN,NaN,Single Family Residential,NaN,NaN,Floor/Wall
7,17177301,0.1638,2016-01-02,2543293,nan,nan,NaN,2.5,3.0,nan,...,nan,NaN,6.111003e+13,NaN,NaN,NaN,Condominium,NaN,NaN,NaN
8,14739064,-0.0030,2016-01-02,519006,nan,nan,NaN,1.0,2.0,nan,...,nan,NaN,6.059042e+13,NaN,NaN,NaN,Condominium,NaN,NaN,NaN
9,14677559,0.0843,2016-01-03,1604540,nan,nan,NaN,2.0,2.0,nan,...,nan,NaN,6.059063e+13,NaN,NaN,NaN,Condominium,NaN,NaN,NaN


In [63]:
def missing_values_col(df):
    '''
    Write or use a previously written function to return the
    total missing values and the percent missing values by column.
    '''
    null_count = df.isnull().sum()
    null_percentage = (null_count / df.shape[0]) * 100
    empty_count = pd.Series(((df == ' ') | (df == '')).sum())
    empty_percentage = (empty_count / df.shape[0]) * 100
    nan_value = pd.Series(((df == 'nan') | (df == 'NaN')).sum())
    nan_percentage = (nan_value / df.shape[0]) * 100
    return pd.DataFrame({'num_missing': null_count, 'missing_percentage': null_percentage,
                         'num_empty': empty_count, 'empty_percentage': empty_percentage,
                         'nan_value': nan_value, 'nan_percentage': nan_percentage})


missing_values_col(df)

,num_missing,missing_percentage,num_empty,empty_percentage,nan_value,nan_percentage
parcelid,0,0.000000,0,0.0,0,0.000000
logerror,0,0.000000,0,0.0,0,0.000000
transactiondate,0,0.000000,0,0.0,0,0.000000
id,0,0.000000,0,0.0,0,0.000000
airconditioningtypeid,0,0.000000,0,0.0,112271,67.640467
architecturalstyletypeid,0,0.000000,0,0.0,165514,99.718042
basementsqft,165892,99.945777,0,0.0,0,0.000000
bathroomcnt,0,0.000000,0,0.0,0,0.000000
bedroomcnt,0,0.000000,0,0.0,0,0.000000
buildingclasstypeid,0,0.000000,0,0.0,165966,99.990360


In [66]:
# cutoff: columns that have 50% or more null values will be removed
df = df.drop(columns=['airconditioningtypeid', 'architecturalstyletypeid', 'buildingclasstypeid',
                      'decktypeid', 'basementsqft', 'hashottuborspa', 'storydesc', 'typeconstructiondesc',
                      'storytypeid', 'typeconstructiontypeid', 'taxdelinquencyflag',])
df = df.drop(columns=['finishedfloor1squarefeet', 'finishedsquarefeet13', 'finishedsquarefeet15',
             'finishedsquarefeet50', 'finishedsquarefeet6', 'fireplacecnt', 'garagecarcnt',
             'garagetotalsqft', 'threequarterbathnbr', 'yardbuildingsqft17', 'yardbuildingsqft26',
             'numberofstories', 'taxdelinquencyyear', 'airconditioningdesc',
             'architecturalstyledesc', 'buildingclassdesc'])

In [69]:
df = df.drop(columns=['poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
                      'regionidneighborhood', 'fireplaceflag'])

In [74]:
missing_values_col(df)

,num_missing,missing_percentage,num_empty,empty_percentage,nan_value,nan_percentage
parcelid,0,0.000000,0,0.0,0,0.000000
logerror,0,0.000000,0,0.0,0,0.000000
transactiondate,0,0.000000,0,0.0,0,0.000000
id,0,0.000000,0,0.0,0,0.000000
bathroomcnt,0,0.000000,0,0.0,0,0.000000
bedroomcnt,0,0.000000,0,0.0,0,0.000000
buildingqualitytypeid,0,0.000000,0,0.0,59043,35.571929
calculatedbathnbr,115,0.069285,0,0.0,0,0.000000
calculatedfinishedsquarefeet,27,0.016267,0,0.0,0,0.000000
finishedsquarefeet12,6721,4.049234,0,0.0,0,0.000000


In [75]:
# remove the rows or columns of any that cannot be reasonably imputed.
df = df.drop(columns=['buildingqualitytypeid', 'heatingorsystemtypeid', 'propertyzoningdesc',
                      'unitcnt', 'heatingorsystemdesc'])

In [76]:
missing_values_col(df)

,num_missing,missing_percentage,num_empty,empty_percentage,nan_value,nan_percentage
parcelid,0,0.000000,0,0.0,0,0.000000
logerror,0,0.000000,0,0.0,0,0.000000
transactiondate,0,0.000000,0,0.0,0,0.000000
id,0,0.000000,0,0.0,0,0.000000
bathroomcnt,0,0.000000,0,0.0,0,0.000000
bedroomcnt,0,0.000000,0,0.0,0,0.000000
calculatedbathnbr,115,0.069285,0,0.0,0,0.000000
calculatedfinishedsquarefeet,27,0.016267,0,0.0,0,0.000000
finishedsquarefeet12,6721,4.049234,0,0.0,0,0.000000
fips,0,0.000000,0,0.0,0,0.000000


In [72]:
def missing_values_row(df):
    null_count = df.isnull().sum(axis=1)
    null_percentage = (null_count / df.shape[1]) * 100
    return pd.DataFrame({'num_missing': null_count, 'percentage': null_percentage})

missing_values_row(df)

,num_missing,percentage
0,0,0.000000
1,4,11.764706
2,0,0.000000
3,0,0.000000
4,3,8.823529
5,0,0.000000
6,0,0.000000
7,4,11.764706
8,4,11.764706
9,4,11.764706


In [ ]:
def fill_with_zeroes(df, *cols):
    for col in cols:
        df[col] = df[col].fillna(0)
    return df

fill_with_zeroes(df, '')

In [ ]:
# cutoff: columns that have 50% or more null values will be removed
df = df.drop(columns=['basementsqft', '', '',
                      '', '', '',
                      '', '', '',
                      '', '', '',
                      '', '', '',])

In [36]:
df.shape

(165982, 52)

In [41]:
missing_values_col(df)

,num_missing,percentage,num_empty,nan_value
parcelid,0,0.000000,0,0
logerror,0,0.000000,0,0
transactiondate,0,0.000000,0,0
id,0,0.000000,0,0
airconditioningtypeid,0,0.000000,0,112271
architecturalstyletypeid,0,0.000000,0,165514
basementsqft,165892,99.945777,0,0
bathroomcnt,0,0.000000,0,0
bedroomcnt,0,0.000000,0,0
buildingclasstypeid,0,0.000000,0,165966
